# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/viveknatan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/viveknatan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - and download our webpages which we'll be using for our data today.

These webpages are from [Simon Willison's](https://simonwillison.net/) yearly "AI learnings".

- [2023 Blog](https://simonwillison.net/2023/Dec/31/ai-in-2023/)
- [2024 Blog](https://simonwillison.net/2024/Dec/31/llms-in-2024/)

Let's start by collecting our data into a useful pile!

In [5]:
!mkdir data

mkdir: data: File exists


In [6]:
!curl https://simonwillison.net/2023/Dec/31/ai-in-2023/ -o data/2023_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 31528    0 31528    0     0  43233      0 --:--:-- --:--:-- --:--:-- 43189


In [7]:
!curl https://simonwillison.net/2024/Dec/31/llms-in-2024/ -o data/2024_llms.html

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70695    0 70695    0     0   507k      0 --:--:-- --:--:-- --:--:--  511k


Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [8]:
from langchain_community.document_loaders import DirectoryLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.html")
docs = loader.load()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [9]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/viveknatan/Documents/AIMCourse/sdg/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/viveknatan/Documents/AIMCourse/sdg/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/viveknatan/Documents/AIMCourse/sdg/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [10]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [11]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 2, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [12]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/18 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 10, relationships: 26)

We can save and load our knowledge graphs as follows.

In [13]:
kg.save("ai_across_years_kg.json")
ai_across_years_kg = KnowledgeGraph.load("ai_across_years_kg.json")
ai_across_years_kg

KnowledgeGraph(nodes: 10, relationships: 26)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=ai_across_years_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [15]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

SingleHopSpecificQuerySynthesizer - Generates simple, direct questions that can be answered looking at a single piece of information. They are straightforward questions that do not require connecting different pieces of information

MultiHopAbstractQuerySynthesizer - Creates questions that require connecting multiple pieces of information to form a more abstract or conceptual answer. The questions often deal with themese, concepts and not just facts.

MultiHopSpecificQuerySynthesizer - Generates questions that require connecting specific facts or details across multiple sources. The se questions combine specific details from different parts of the knowledge graph.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [17]:
# testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset = generator.generate(testset_size=25, query_distribution=query_distribution)
testset.to_pandas()

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/28 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,Considering the role of Meta in the developmen...,[The ethics of this space remain diabolically ...,Meta has played a significant role in making l...,single_hop_specifc_query_synthesizer
1,What is TII in AI research?,[The ethics of this space remain diabolically ...,TII in Abu Dhabi is mentioned as one of the or...,single_hop_specifc_query_synthesizer
2,Who is Simon Willison and what did he say abou...,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s Weblog discusses that 2023 wa...,single_hop_specifc_query_synthesizer
3,Considering the advancements in Artificial Int...,[Simon Willison’s Weblog Subscribe Stuff we fi...,"In 2023, Large Language Models (LLMs) represen...",single_hop_specifc_query_synthesizer
4,What is Stanford Alpaca in context of large la...,[the document includes some of the clearest ex...,Stanford Alpaca is mentioned in the context of...,single_hop_specifc_query_synthesizer
5,What is DALL-E?,[the document includes some of the clearest ex...,The provided context does not include a specif...,single_hop_specifc_query_synthesizer
6,Wha is Google's role in de development of larg...,[Everything tagged “llms” on my blog in 2024 T...,"Google's Gemini 1.5 Pro, released in February,...",single_hop_specifc_query_synthesizer
7,"As an AI Research and Development Manager, how...",[Everything tagged “llms” on my blog in 2024 T...,The provided context does not include specific...,single_hop_specifc_query_synthesizer
8,How does OpenAI's pricing and model developmen...,[collapse in the cost of running a prompt thro...,OpenAI's pricing has significantly decreased f...,single_hop_specifc_query_synthesizer
9,What is Gemini 1.5 Flash 8B?,[collapse in the cost of running a prompt thro...,Gemini 1.5 Flash 8B is a model from Google tha...,single_hop_specifc_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [20]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/2 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/2 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/12 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'StringIO' object has no attribute 'output'


Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [21]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What is Stability AI's role in the development...,[My blog in 2023 Here’s the sequel to this pos...,Stability AI is mentioned among the organizati...,single_hop_specifc_query_synthesizer
1,Does December affect the performance of LLMs l...,[openly licensed ones are still the world’s mo...,The context mentions the question of whether C...,single_hop_specifc_query_synthesizer
2,AI what is it,[Simon Willison’s Weblog Subscribe Stuff we fi...,Simon Willison’s weblog states that Artificial...,single_hop_specifc_query_synthesizer
3,What is DALL-E 3 and how does it relate to the...,"[of what LLMs are, how they work and how they ...",The provided context does not include specific...,single_hop_specifc_query_synthesizer
4,Considering the significant capabilities of La...,[<1-hop>\n\nopenly licensed ones are still the...,Developers can harness the strengths of Large ...,multi_hop_abstract_query_synthesizer
5,H0w do increased contex length and input capab...,[<1-hop>\n\nEverything tagged “llms” on my blo...,The context segments explain that models like ...,multi_hop_abstract_query_synthesizer
6,H0w have advancments in LLMs and model efficie...,[<1-hop>\n\nEverything tagged “llms” on my blo...,"In 2024, advancements in Large Language Models...",multi_hop_abstract_query_synthesizer
7,how AI agents and personal assistants are deve...,[<1-hop>\n\nopenly licensed ones are still the...,The context explains that AI agents and person...,multi_hop_abstract_query_synthesizer
8,How do ChatGPT and other LLMs demonstrate thei...,[<1-hop>\n\nopenly licensed ones are still the...,The context highlights that ChatGPT and other ...,multi_hop_specific_query_synthesizer
9,"How does ChatGPT's capabilites, like voice mod...","[<1-hop>\n\nNotebookLM, released in September,...",The context describes recent advancements in C...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [22]:
from langsmith import Client

client = Client()

dataset_name = "State of AI Across the Years!"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="State of AI Across the Years!"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [23]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [24]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [26]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [27]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="State of AI"
)

/Users/viveknatan/Documents/AIMCourse/sdg/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:758: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is empty, alternative syntax for `is_empty: \&quot;field_name\&quot;`",
/Users/viveknatan/Documents/AIMCourse/sdg/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/qdrant_client/http/models/models.py:762: SyntaxWarning: invalid escape sequence '\&'
  description="Check that the field is null, alternative syntax for `is_null: \&quot;field_name\&quot;`",


In [28]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [29]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [30]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [31]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [32]:
rag_chain.invoke({"question" : "What are Agents?"})

'Based on the provided context, "agents" is an extremely vague and poorly defined term in the AI field. Generally, it seems to refer to AI systems that can act on your behalf, akin to a travel agent model, or Large Language Models (LLMs) that have access to tools which they can use iteratively to solve problems. However, there is no single clear or widely accepted definition of "agents," and the concept remains more of a promising idea than a fully realized technology in production. Additionally, the utility of agents is questioned due to challenges like gullibility, where AI models tend to believe everything they are told, making meaningful autonomous decision-making difficult without achieving Artificial General Intelligence (AGI). Overall, "agents" are AI systems designed to act autonomously for users, but the field lacks clarity and mature implementations.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [33]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [34]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dope_or_nope_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this submission dope, lit, or cool?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`: checks if the model's response correctly answers the given question. It looks at the model's output and compares it against the reference answer to determine if the response is factually accurate
- `labeled_helpfulness_evaluator`: assesses how helpful the response is to the user. It takes into account both the model's output and the correct reference answer
- `dope_or_nope_evaluator`: subjective evaluator that assesses the "coolness" or style of the response. It's designed to measure how engaging and well-styled the responses are, beyond just factual accuracy.

## LangSmith Evaluation

In [35]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'glossy-hobbies-86' at:
https://smith.langchain.com/o/c9d5843a-35e9-4d41-83e6-32315ffaf39f/datasets/a783f1c0-a4fe-47fc-b46f-6b6bc451f419/compare?selectedSessions=11c2de19-b888-4b40-ad04-77eacc9948de




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Considering the developments in Large Language...,"Based on the provided context, the advancement...",None,"In 2024, significant progress was made with in...",1,1,0,6.239416,e8dbe2e3-384a-4315-8c70-c0b0cf611c90,dd832817-9888-49f3-89c1-ced7bf2c9e04
1,Considering the advancements in Meta's large l...,"Based on the provided context, Meta’s developm...",None,Meta's development of models such as Llama 3.2...,1,1,0,4.703618,9a17246a-ddbe-4834-9423-21793a71f4ad,e130a4cb-6d23-4406-83db-6c8900dc9c5f
2,"How does ChatGPT's capabilites, like voice mod...","Based on the provided context, ChatGPT's capab...",None,The context describes recent advancements in C...,1,1,0,4.091940,c9e87463-7191-4583-993b-c4d7f1cc481c,a0250e79-53a7-471d-bc1a-85e78d222743
3,How do ChatGPT and other LLMs demonstrate thei...,ChatGPT and other LLMs demonstrate their capab...,None,The context highlights that ChatGPT and other ...,1,1,0,3.138001,3253af5f-03ec-4924-b2e9-e69118c7835b,7e51932f-8c37-47f4-a15c-f6d1c4514472
4,how AI agents and personal assistants are deve...,Based on the provided context:\n\nAI agents ar...,None,The context explains that AI agents and person...,1,1,0,4.303455,4ab3b532-8ca2-483a-a08f-3557a91c5e8b,bff06ab4-7744-4669-bcbc-ee8cd20dd3d3
5,H0w have advancments in LLMs and model efficie...,"Based on the provided context, the advancement...",None,"In 2024, advancements in Large Language Models...",1,0,0,6.378394,39bc8172-5f91-413a-823b-695b4e1cee12,3a8492a2-8b69-4166-a962-fb602eb680c2
6,H0w do increased contex length and input capab...,Increased context length allows LLMs to handle...,None,The context segments explain that models like ...,1,1,0,2.919201,2cb6601d-3474-4660-ae43-7dd403502207,c11c0645-1703-4f86-9e47-f3f06a7f2a5f
7,Considering the significant capabilities of La...,"Based on the provided context, developers can ...",None,Developers can harness the strengths of Large ...,1,1,0,4.653347,8f7e42ff-7bb5-4579-ba37-a55cea849157,4cddbc97-4ccf-4355-b371-057ba3f75395
8,What is DALL-E 3 and how does it relate to the...,I don't know.,None,The provided context does not include specific...,1,0,0,0.917808,9dbacb10-a2db-497f-8e83-814ccb36700e,ed63933a-baa0-4291-9655-259657736514
9,AI what is it,"Based on the provided context, AI, particularl...",None,Simon Willison’s weblog states that Artificial...,1,0,0,3.855883,562efb1f-22cc-4f98-868e-e35f315cfc54,45fd3e0f-8b3f-4f1d-91d4-6a62703eaca3


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [36]:
DOPE_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the questions in a dope way, be cool!

Context: {context}
Question: {question}
"""

dope_rag_prompt = ChatPromptTemplate.from_template(DOPE_RAG_PROMPT)

In [37]:
rag_documents = docs

In [38]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?
The RAG app would have more context to base its answer on. It is more likely to contain the complete answer and also broader context. The chunk size modification helps balance between having enough context for comprehensive answers while maintaining efficient retrieval and processing.

In [39]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?
The embedding model change to text-embedding-3-large improves the application's performance by providing more accurate semantic understanding and better context retrieval, leading to more relevant information being passed to the LLM for generating answers. This results in higher quality responses because the model can better understand and match the semantic relationships between queries and content, ultimately improving the accuracy and relevance of the system's outputs.

In [40]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="AI Across Years (Augmented)"
)

In [41]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [42]:
dope_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dope_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [43]:
dope_rag_chain.invoke({"question" : "what are Agents?"})

'Alright, here’s the lowdown on "agents"—it’s a term that’s all over the place and kinda vague, no cap. People drop "agents" like it’s some magic AI that just does your bidding, but really, it means different things to different folks. \n\nSome folks see agents as AI that acts *on your behalf*—think travel agent vibes, booking stuff, making calls. Others think of agents as LLMs (big language models) hooked up with tools to solve problems in a loop. Toss in "autonomy," and it gets messier since nobody nails down what that means exactly.\n\nThe kicker? Real, useful agents that don’t get fooled or spout nonsense haven’t arrived yet. The main hitch is gullibility—AI believing everything it reads, like that viral bogus “Encanto 2” movie summary Google accidentally served up. Until agents can tell real from fake consistently (which might need that mythical AGI), they’re more “coming soon” than reality.\n\nSo, TL;DR: Agents = loosely defined AI systems that act for you, but nobody’s cracked t

Finally, we can evaluate the new chain on the same test set!

In [44]:
evaluate(
    dope_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dope_or_nope_evaluator
    ],
    metadata={"revision_id": "dope_chain"},
)

View the evaluation results for experiment: 'definite-knife-98' at:
https://smith.langchain.com/o/c9d5843a-35e9-4d41-83e6-32315ffaf39f/datasets/a783f1c0-a4fe-47fc-b46f-6b6bc451f419/compare?selectedSessions=ec0ced37-9d0f-4355-8df7-4b2f5b912873




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,Considering the developments in Large Language...,"Alright, here’s the lowdown, fresh and fly fro...",None,"In 2024, significant progress was made with in...",1,1,1,5.157956,e8dbe2e3-384a-4315-8c70-c0b0cf611c90,4a951427-4749-46c9-b791-45b67f41c72a
1,Considering the advancements in Meta's large l...,"Alright, here’s the lowdown, fresh and fly: Me...",None,Meta's development of models such as Llama 3.2...,1,1,1,4.192485,9a17246a-ddbe-4834-9423-21793a71f4ad,d0e6de3d-06b8-42e0-b3c4-bac6d6c1e46f
2,"How does ChatGPT's capabilites, like voice mod...","Yo, here’s the lowdown on ChatGPT’s latest vib...",None,The context describes recent advancements in C...,1,1,1,5.050800,c9e87463-7191-4583-993b-c4d7f1cc481c,82164459-983e-464c-b9d2-5e551c1884d8
3,How do ChatGPT and other LLMs demonstrate thei...,"Alright, here’s the lowdown on how ChatGPT and...",None,The context highlights that ChatGPT and other ...,1,1,1,5.820432,3253af5f-03ec-4924-b2e9-e69118c7835b,3a99695a-1769-46c2-9879-02e9be583615
4,how AI agents and personal assistants are deve...,"Alright, here’s the lowdown on AI agents, pers...",None,The context explains that AI agents and person...,1,1,1,4.259145,4ab3b532-8ca2-483a-a08f-3557a91c5e8b,aa2f358f-ac0a-401c-a5a8-659f695906f8
5,H0w have advancments in LLMs and model efficie...,"Yo, here’s the lowdown on 2024’s LLM scene, st...",None,"In 2024, advancements in Large Language Models...",1,0,1,5.115815,39bc8172-5f91-413a-823b-695b4e1cee12,08979c9c-1802-42fc-bc87-862716b35128
6,H0w do increased contex length and input capab...,"Yo, here’s the lowdown—longer context lengths ...",None,The context segments explain that models like ...,1,1,1,3.250598,2cb6601d-3474-4660-ae43-7dd403502207,66c6ffa8-7339-4148-ba42-e91d6f984417
7,Considering the significant capabilities of La...,"Alright, let’s break it down and keep it smoot...",None,Developers can harness the strengths of Large ...,1,1,1,4.808369,8f7e42ff-7bb5-4579-ba37-a55cea849157,e398396d-f143-44f1-9715-6da2ab287839
8,What is DALL-E 3 and how does it relate to the...,"Alright, here’s the lowdown on DALL-E 3 from t...",None,The provided context does not include specific...,0,0,1,3.574658,9dbacb10-a2db-497f-8e83-814ccb36700e,854a1eec-9ee5-4ab7-9d64-50944faadc48
9,AI what is it,"Yo, AI in this context is basically those Larg...",None,Simon Willison’s weblog states that Artificial...,1,1,1,2.618729,562efb1f-22cc-4f98-868e-e35f315cfc54,21274378-4eeb-4401-a96f-5a7d94781683


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

![LangChain screen showing difference bwetween the original and enhanced chains](Untitled.png)

Correctness: The enhanced chain ended up getting 2 answers factually incorrect which likely happened as we were focussed on expanding context and making the response cooler.

Helpfulness: The overall helpfulness did not change between the two chains as this aspect wasnt comprimised with any of the changes

Dopeness: Increased significantly as we instructed the LLM to respond in a cool fashion. 

An improvement in one metric might come at the cost of another (in this case dopeness vs correctness), demonstrating the importance of carefully balancing different aspects of response quality based on the specific use case and audience.